# Connecting to free language learing model APIs

## 1. Co:here

In [1]:
# import libraries
import pandas as pd
import cohere
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)

In [11]:
# get the data
df = pd.read_excel("../data/Example_data.xlsx")

In [13]:
# look at data
df.head(1)

Domain  \
0  rassegnastampa.news   

                                                                                                                       Title  \
0  Boris Johnson using a taxpayer-funded jet for an election campaign fits a long history of taking things he didn't pay for   

                                                                                                                                                                                                                                                   Description  \
0  …often trigger a protest vote that can upset…that ministers who breach the Ministerial Code…plane up the M1 in order to…serious questions about abuse of taxpayers' money which is a clear breach of the code.…should not permit abuse of taxpayers' money…   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [14]:
df.shape

(10, 9)

In [17]:
# df.index()

In [20]:
# create a sample dataset
num_examples = 5
df_sample = df.sample(num_examples)

In [22]:
# split into training and testing sets
# we set the 'Body' column as the prompt and the 'Analyst_Average_Score' col as the target variable
sentences_train, sentences_test, labels_train, labels_test = train_test_split(list(df_sample['Body']), 
                                                                list(df_sample['Analyst_Average_Score']), 
                                                                test_size=0.25, 
                                                                random_state=0)

### Getting the embeddings of the item content

In [23]:
# connecting to the api
api_key = "i5BJ67vhgp95kK7G6RSxEKVpjbeRJUwIufURVEnX"

# creating and retrieving a cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [24]:
# embed the training set
embeddings_train = co.embed(texts=sentences_train,
                             model="large",
                             truncate="LEFT").embeddings
# embed the testing set
embeddings_test = co.embed(texts=sentences_test,
                             model="large",
                             truncate="LEFT").embeddings

We have not created 2 sets of embeddings:
<li> embeddings_train - contains embeddings of the training sentences
<li> embeddings_test - contains embeddings of the testing sentences

In [25]:
# to see what an embedding looks like
print(f"Review text: {sentences_train[0]}")
print(f"Embedding vector: {embeddings_train[0][:10]}")

Review text: - Former Eskom CEO Matshela Moses Koko sought R500 000 in damages and an apology from a 72-year-old woman. - Barbara Tanton responded on Twitter to Koko's tweet about a City Press article. - The judge dismissed the application as an "abuse of court" and awarded punitive costs against Koko. The 72-year-old principal of a Gauteng pre-school successfully fought off an attempt by former Eskom CEO Matshela Moses Koko to seek R500 000 in damages for a tweet she posted about him. Koko sought an apology, a restriction, an interdict and R500 000 in damages for defamation from Barbara Tanton. Tanton's lawyers said his legal action was nothing more than a SLAPP suit (Strategic Lawsuit Against Public Participation) designed to intimidate her. Johannesburg High Court Acting Judge Adele de Wet agreed. She dismissed the application and ordered Koko to pay costs on a punitive scale. The judge said Koko's motive was not to redress any wrong, but to "punish" Tanton. "Such conduct is vexatio

### Training a classifier using the training set

In [28]:
# import SVM classifier code
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# Initialize a support vector machine, with class_weight='balanced' because 
# our training set has roughly an equal amount of positive and negative 
# sentiment sentences
svm_classifier = make_pipeline(StandardScaler(), SVC(class_weight='balanced')) 

# fit the support vector machine
svm_classifier.fit(embeddings_train, labels_train)

ValueError: Unknown label type: 'continuous'

### Evaluate the performance of the classifier on the testing set

In [27]:
# get the score from the test set, and print it out to screen!
score = svm_classifier.score(embeddings_test, labels_test)
print(f"Validation accuracy on Large is {100*score}%!")

AttributeError: 'SVC' object has no attribute 'support_vectors_'